In [1]:
import tensorflow as tf
import numpy as np

In [2]:
class FMLayer(tf.keras.layers.Layer):
    '''
    自定义FM层
    '''
    def __init__(self,k):
        super(FMLayer,self).__init__()
        
        self.linear_part=tf.keras.layers.Dense(1,activation=None)

        self.k=k
    
    def build(self,input_shape):
        '''
        input_shape为tf自动获取
        '''
        # 矩阵V的形状为(n_features, k)
        self.v=self.add_weight(shape=(input_shape[-1],self.k),initializer=tf.random_normal_initializer(), trainable=True)
    
    def call(self,inputs):
        linear=self.linear_part(inputs)
        
        wide=0.5*tf.reduce_sum(tf.pow(tf.matmul(inputs,self.v),2)-
                                   tf.matmul(tf.pow(inputs,2),tf.pow(self.v,2)),
                                    axis=1,keepdims=True)
        return linear+wide

class FactorizationMachine(tf.keras.Model):
    def __init__(self,k):
        super(FactorizationMachine,self).__init__()
        
        self.fm_layer=FMLayer(k)
        
    def call(self,inputs):
        return self.fm_layer(inputs)

In [3]:
x_data = np.matrix([
#    Users  |     Movies     |    Movie Ratings   | Time | Last Movies Rated
#   A  B  C | TI  NH  SW  ST | TI   NH   SW   ST  |      | TI  NH  SW  ST
    [1, 0, 0,  1,  0,  0,  0,   0.3, 0.3, 0.3, 0,     13,   0,  0,  0,  0 ],
    [1, 0, 0,  0,  1,  0,  0,   0.3, 0.3, 0.3, 0,     14,   1,  0,  0,  0 ],
    [1, 0, 0,  0,  0,  1,  0,   0.3, 0.3, 0.3, 0,     16,   0,  1,  0,  0 ],
    [0, 1, 0,  0,  0,  1,  0,   0,   0,   0.5, 0.5,   5,    0,  0,  0,  0 ],
    [0, 1, 0,  0,  0,  0,  1,   0,   0,   0.5, 0.5,   8,    0,  0,  1,  0 ],
    [0, 0, 1,  1,  0,  0,  0,   0.5, 0,   0.5, 0,     9,    0,  0,  0,  0 ],
    [0, 0, 1,  0,  0,  1,  0,   0.5, 0,   0.5, 0,     12,   1,  0,  0,  0 ]
])
# ratings
y_data = np.array([5, 3, 1, 4, 5, 1, 5])

# Let's add an axis to make tensoflow happy.
y_data.shape += (1, )

In [4]:
fm=FactorizationMachine(5)

optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)

fm.compile(optimizer=optimizer,loss=tf.keras.losses.MeanSquaredError())

hist=fm.fit(x_data,y_data,epochs=5)

Epoch 1/5
1/1 [==============================] - 0s 1ms/step - loss: 5.8485
Epoch 2/5
1/1 [==============================] - 0s 0s/step - loss: 5.1890
Epoch 3/5
1/1 [==============================] - 0s 1ms/step - loss: 4.6709
Epoch 4/5
1/1 [==============================] - 0s 1ms/step - loss: 4.2156
Epoch 5/5
1/1 [==============================] - 0s 1ms/step - loss: 3.8064


In [5]:
fm.predict(x_data)

array([[3.4668858],
       [3.652246 ],
       [4.41393  ],
       [2.1322427],
       [3.3418107],
       [2.2920125],
       [3.6451492]], dtype=float32)